# Load librabies

In [14]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import random

# Load Data


In [24]:
# Load the graph and the deleted edge information from the file
with open("../../graphs_and_deleted_edges.pickle", "rb") as f:
    all_graphs, all_deleted_edges = pickle.load(f)

In [25]:
print(len(all_graphs), len(all_deleted_edges))

12 12


In [27]:
all_deleted_edges[:3]

[(tensor(1),
  tensor(0),
  {'src_port': tensor([3]), 'dst_port': tensor([3]), 'edge_type': tensor([0])}),
 (tensor(1),
  tensor(0),
  {'src_port': tensor([2]), 'dst_port': tensor([2]), 'edge_type': tensor([0])}),
 (tensor(1),
  tensor(0),
  {'src_port': tensor([2]), 'dst_port': tensor([2]), 'edge_type': tensor([0])})]

In [21]:
# Split the data into training and validation sets
train_graphs = []
train_deleted_edges = []
val_graphs = []
val_deleted_edges = []

for graph, deleted_edge in zip(all_graphs, all_deleted_edges):
    if random.random() < 0.8:
        train_graphs.append(graph)
        train_deleted_edges.append(deleted_edge)
    else:
        val_graphs.append(graph)
        val_deleted_edges.append(deleted_edge)


In [22]:
print(len(train_graphs), len(train_deleted_edges))

11 11


# Training